In [26]:
import numpy as np

class PhaseEquilibriumPR:
    def __init__(self, components, mole_fraction, kij=None):
        self.components = list(components.keys())
        self.mole_fraction = np.array(mole_fraction)
        self.n = len(self.components)
        self.R = 8.314
        
        self.Tc = np.array([components[comp]['Tc'] for comp in self.components])
        self.Pc = np.array([components[comp]['Pc'] for comp in self.components])
        self.omega = np.array([components[comp]['omega'] for comp in self.components])
        self.Mw = np.array([components[comp]['Mw'] for comp in self.components])
        
        self.kij = np.zeros((self.n, self.n)) if kij is None else np.array(kij)
        
        self.ai = np.zeros(self.n)
        self.bi = np.zeros(self.n)
        self.alpha_i = np.zeros(self.n)

    def CoeffPR(self, T):
        self.ai = 0.45724 * (self.R**2 * self.Tc**2) / self.Pc
        self.bi = 0.07780 * self.R * self.Tc / self.Pc

        mi = np.where(self.omega <= 0.49,
                    0.37464 + 1.54226 * self.omega - 0.26992 * self.omega**2,
                    0.3796 + 1.485 * self.omega - 0.1644 * self.omega**2 + 0.01667 * self.omega**3)
        
        self.alpha_i = (1 + mi * (1 - np.sqrt(T / self.Tc)))**2

    def MixParam(self, z):
        Amix = 0
        for i in range(self.n):
            for j in range(self.n):
                Amix += z[i] * z[j] * np.sqrt(self.ai[i] * self.ai[j]) * (1 - self.kij[i,j])
        Bmix = np.sum(z * self.bi)
        return Amix, Bmix

    def EqSolve(self, a, b, P, T):
        A = a * P / (self.R * T)**2
        B = b * P / (self.R * T)
        
        coeffs = [1, -(1 - B), (A - 3 * B**2 - 2 * B), -(A * B - B**2 - B**3)]
        roots = np.roots(coeffs)
        real_roots = np.real(roots[np.isreal(roots)])
        return real_roots[real_roots > 0]

    def Volatility(self, x, phase, P, T):
        Amix, Bmix = self.MixParam(x)
        roots = self.EqSolve(Amix, Bmix, P, T)
        
        if len(roots) == 0:
            return np.full(self.n, np.nan), np.nan, np.nan, np.nan
           
        if phase == 'liquid':
             V = np.min(roots)
        else:
            V = np.max(roots)
           
        Z = P * V / (self.R * T)
        A = Amix * P / (self.R * T)**2
        B = Bmix * P / (self.R * T)

        for i in range(self.n):
            summ = 0
            for j in range(self.n):
                summ += x[j] * np.sqrt(self.ai[i] * self.ai[j]) * (1 - self.kij[i,j])

        t1 = self.bi / Bmix * (Z - 1)
        t2 = -np.log(Z - B)
        t3 = -A / (2 * np.sqrt(2) * B) * (2 * summ / Amix - self.bi[i] / Bmix)
        t3 *= np.log((Z + (1 + np.sqrt(2)) * B) / (Z + (1 - np.sqrt(2)) * B))
            
        phi = np.exp(t1 + t2 + t3)
        rho = np.sum(x * self.Mw) * 1e-3 / V
            
        return phi, Z, V, rho

    def StabilityCheck(self, z, P, T, phase):
        
        phi_z, _, _, _ = self.Volatility(z, phase, P, T)
            
        if phase == 'liquid':
            y_test = z * np.exp(np.random.normal(0, 0.1, size = len(z)))
            y_test /= np.sum(y_test)
            phi_y, _, _, _ = self.Volatility(y_test, 'vapor', P, T)
            R = np.sum(y_test * (np.log(y_test + 1e-100) + np.log(phi_y + 1e-100) - 
                        np.log(z + 1e-100) - np.log(phi_z + 1e-100)))
        else:
            x_test = z * np.exp(np.random.normal(0, 0.1, size = len(z)))
            x_test /= np.sum(x_test)
            phi_x, _, _, _ = self.Volatility(x_test, 'liquid', P, T)
            R = np.sum(x_test * (np.log(x_test + 1e-100) + np.log(phi_x + 1e-100) - 
                       np.log(z + 1e-100) - np.log(phi_z + 1e-100)))
        
        return R >= 0

    def SearchEquilibrium(self, P, T, max_iter = 500):
        
        self.CoeffPR(T)

        #if self.StabilityCheck(self.mole_fraction, P, T, 'liquid'):
            #return self.GetSingleResult(P, T, 'liquid')
        
        #if self.StabilityCheck(self.mole_fraction, P, T, 'vapor'):
            #return self.GetSingleResult(P, T, 'vapor')
        
        K = self.Pc / P * np.exp(5.373 * (1 + self.omega) * (1 - self.Tc / T))
        x = self.mole_fraction
        y = K * x
        
        x = x / np.sum(x)
        y = y / np.sum(y)
        
        for iteration in range(max_iter):
            phi_liq, _, _, _ = self.Volatility(x, 'liquid', P, T)
            phi_vap, _, _, _ = self.Volatility(y, 'vapor', P, T)
                
            K_new = phi_liq / phi_vap
            
            error = np.sum(np.abs(K_new - K))
            
            if error < 1e-12:
                stable_liq = self.StabilityCheck(x, P, T, 'liquid')
                stable_vap = self.StabilityCheck(y, P, T, 'vapor')
                if stable_liq and stable_vap:
                    k_vap = np.clip(np.sum((self.mole_fraction - x)/(y - x + 1e-12))/self.n, 0, 1)
                    return self.GetResult(x, y, k_vap, 'new', P, T, iteration + 1)
                else:
                    pass
            
            K = K_new
            y = K * x
            y = y / np.sum(y)
        
        return self.GetResult(None, None, None, 'failed - max iterations', P, T)

    def GetResult(self, x, y, k_vap, status, P, T, iter = None):
        
        def convert_values(d):
            if isinstance(d, dict):
                return {k: float(v) if isinstance(v, (np.floating, float)) else v for k, v in d.items()}
            return float(d) if isinstance(d, (np.floating, float)) else d
        
        result = {
            'status': status,
            'pressure': float(P),
            'temperature': float(T),
            'phase_fraction': convert_values(k_vap) if k_vap is not None else None
        }
        
        if x is not None:
            phi, Z, V, rho = self.Volatility(x, 'liquid', P, T)
            result['liquid'] = {
                'mole_fraction': convert_values(dict(zip(self.components, x))),
                'volatility': convert_values(dict(zip(self.components, phi))),
                'Z': convert_values(Z),
                'molar_volume': convert_values(V),
                'density': convert_values(rho)
            }
        
        if y is not None:
            phi, Z, V, rho = self.Volatility(y, 'vapor', P, T)
            result['vapor'] = {
                'mole_fraction': convert_values(dict(zip(self.components, y))),
                'volatility': convert_values(dict(zip(self.components, phi))),
                'Z': convert_values(Z),
                'molar_volume': convert_values(V),
                'density': convert_values(rho)
            }
        
        if iter is not None:
            result['iterations'] = int(iter)
            
        return result

    def GetSingleResult(self, P, T, phase):
        props = self.Volatility(self.mole_fraction, phase, P, T)
        result = {
            'status': f'single {phase}',
            'pressure': float(P),
            'temperature': float(T),
            'phase_fraction': 0.0 if phase == 'liquid' else 1.0,
            phase: {
                'mole_fraction': {k: float(v) for k, v in zip(self.components, self.mole_fraction)},
                'volatility': {k: float(v) for k, v in zip(self.components, props[0])},
                'Z': float(props[1]),
                'molar_volume': float(props[2]),
                'density': float(props[3])
            }
        }
        result['liquid' if phase == 'vapor' else 'vapor'] = None
        return result

def PrintResult(result):
    print("ФАЗОВОЕ РАВНОВЕСИЕ: РЕЗУЛЬТАТЫ РАСЧЕТА")
    
    print(f"\nСтатус: {result['status']}")
    print(f"Давление: {result['pressure']/1e5:.2f} бар")
    print(f"Температура: {result['temperature']:.2f} K")
    
    if 'liquid' in result and result['liquid'] is not None:
        print("\nЖИДКАЯ ФАЗА:")
        print(f"Состав: { {k: float(v) for k, v in result['liquid']['mole_fraction'].items()} }")
        print(f"Коэф. летучести: { {k: float(v) for k, v in result['liquid']['volatility'].items()} }")
        print(f"Z-фактор: {float(result['liquid']['Z']):.4f}")
        print(f"Мольный объем: {float(result['liquid']['molar_volume']):.6e} м3/моль")
        print(f"Плотность: {float(result['liquid']['density']):.2f} кг/м3")
    
    if 'vapor' in result and result['vapor'] is not None:
        print("\nПАРОВАЯ ФАЗА:")
        print(f"Состав: { {k: float(v) for k, v in result['vapor']['mole_fraction'].items()} }")
        print(f"Коэф. летучести: { {k: float(v) for k, v in result['vapor']['volatility'].items()} }")
        print(f"Z-фактор: {float(result['vapor']['Z']):.4f}")
        print(f"Мольный объем: {float(result['vapor']['molar_volume']):.6e} м3/моль")
        print(f"Плотность: {float(result['vapor']['density']):.2f} кг/м3")

if __name__ == "__main__":

    components = {
        'Methane': {'Tc': 190.6, 'Pc': 45.99e5, 'omega': 0.011, 'Mw': 16.04},
        'Propane': {'Tc': 369.8, 'Pc': 42.48e5, 'omega': 0.152, 'Mw': 44.10},
        'Hexane': {'Tc': 507.6, 'Pc': 30.25e5, 'omega': 0.301, 'Mw': 86.18}
    }
    
    kij_matrix = [
        [0.0, 0.01, 0.03],
        [0.01, 0.0, 0.02],
        [0.03, 0.02, 0.0]
    ]
    
    system = PhaseEquilibriumPR(components, [0.2, 0.3, 0.5], kij = kij_matrix)
    result = system.SearchEquilibrium(1.01325e5, 300)  # 1 атм, 15°C
    
    PrintResult(result)

ФАЗОВОЕ РАВНОВЕСИЕ: РЕЗУЛЬТАТЫ РАСЧЕТА

Статус: new
Давление: 1.01 бар
Температура: 300.00 K

ЖИДКАЯ ФАЗА:
Состав: {'Methane': 0.2, 'Propane': 0.3, 'Hexane': 0.5}
Коэф. летучести: {'Methane': 3.210671142095899, 'Propane': 2.3649245272875445, 'Hexane': 1.3763783778518899}
Z-фактор: 0.2070
Мольный объем: 5.094355e-03 м3/моль
Плотность: 11.69 кг/м3

ПАРОВАЯ ФАЗА:
Состав: {'Methane': 1.0, 'Propane': 1.3075960376151123e-18, 'Hexane': 2.314428760239137e-50}
Коэф. летучести: {'Methane': 3535939854453524.0, 'Propane': 2.68408042024396e+34, 'Hexane': 7.17207679042959e+67}
Z-фактор: 40.5033
Мольный объем: 9.970239e-01 м3/моль
Плотность: 0.02 кг/м3
